In [24]:
'''
1. a
'''
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
data_raw = pd.read_csv('Frogs_MFCCs.csv', sep=",", header=0)
data = data_raw.iloc[:,:25]
msk = np.random.rand(len(data)) < 0.7
train_data = data[msk]
np_train_data_features = train_data.iloc[:,:22].values
test_data = data[~msk]
np_test_data_features = test_data.iloc[:,:22].values
label_columns = ["Family","Genus","Species"]
np_train_data_targets = {}
np_test_data_targets = {}
for i in range(3):
    temp_np_train_target = train_data.iloc[:,22+i:22+i+1].values.ravel()
    temp_np_test_target = test_data.iloc[:,22+i:22+i+1].values.ravel()
    np_train_data_targets[label_columns[i]] = temp_np_train_target
    np_test_data_targets[label_columns[i]] = temp_np_test_target

In [52]:
'''
1. b(i), (ii)
'''
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
PENALTY_PRAMA_POINTS = 7
SIGMA_PRAMA_POINTS = 20
svm = SVC()
kf = KFold(n_splits=10)
C_range = np.logspace(0, 6, PENALTY_PRAMA_POINTS)
gamma_range = np.linspace(0.1, 2.0, SIGMA_PRAMA_POINTS)
param_grid = dict(C=C_range, gamma=gamma_range)
best_Cs = [] 
best_Sigmas = []
for label in label_columns:
    gridCV = GridSearchCV(estimator=svm, param_grid=param_grid,scoring='accuracy',cv=kf)

    gridCV.fit(np_train_data_features, np_train_data_targets[label])
    print("The best parameters for label %s are %s with a accuracy of %0.2f"
      % (label, gridCV.best_params_, gridCV.best_score_))
    best_Cs.append(gridCV.best_params_['C'])
    best_Sigmas.append(gridCV.best_params_['gamma'])

The best parameters are {'C': 1.0, 'gamma': 1.4} with a score of 0.75
The best parameters are {'C': 10.0, 'gamma': 0.1} with a score of 0.71
The best parameters are {'C': 10.0, 'gamma': 0.1} with a score of 0.69


In [79]:
'''
1. b (ii) coni.
The best parameters are {'C': 1.0, 'gamma': 1.4} with a score of 0.75
The best parameters are {'C': 10.0, 'gamma': 0.1} with a score of 0.71
The best parameters are {'C': 10.0, 'gamma': 0.1} with a score of 0.69
'''
from sklearn.metrics import hamming_loss
# best_Cs = [1, 10, 10]
# best_Sigmas = [1.4, 0.1, 0.1]
test_pred_labels = []
for i in range(len(label_columns)):
    label = label_columns[i]
    best_C = best_Cs[i]
    best_Sigma = best_Sigmas[i]
    svm = SVC(C=best_C, gamma = best_Sigma)
    svm.fit(np_train_data_features, np_train_data_targets[label])
    temp_score = svm.score(np_test_data_features, np_test_data_targets[label])
    test_pred_labels.append(svm.predict(np_test_data_features))
    print("Test accuracy for label %s is %0.4f" % (label, temp_score))

test_sample_count = len(test_pred_labels[0])
false_count = 0
for i in range(test_sample_count):
    all_true = True 
    for j in range(len(label_columns)):
        label = label_columns[j]
        if(test_pred_labels[j][i] != np_test_data_targets[label][i]):
            all_true = False
    if not all_true:
        false_count += 1
temp_test_targets = []
for label in label_columns:
    temp_test_targets.append (np_test_data_targets[label])
hamming_loss_rate = np.sum(np.not_equal(np.transpose(temp_test_targets),
                        np.transpose(test_pred_labels)))/float(np.transpose(temp_test_targets).size)
print ("Exact match rate is %0.4f" % ((test_sample_count-false_count)/test_sample_count))
print("hamming loss rate is %0.4f" % (hamming_loss_rate))

Test accuracy for label Family is 0.9895
Test accuracy for label Genus is 0.9748
Test accuracy for label Species is 0.9789
Exact match rate is 0.9702
hamming loss rate is 0.0190


In [90]:
'''
1 b (iii)
'''
import warnings
from sklearn.svm import LinearSVC

warnings.filterwarnings("ignore", category=Warning)
PENALTY_PRAMA_POINTS = 10
C_range = np.logspace(-3, 6, PENALTY_PRAMA_POINTS)
linearSVC = LinearSVC(penalty='l1', dual=False, tol=1e-3, max_iter = 100)
kf = KFold(n_splits=10)
param_grid = dict(C=C_range)
best_Cs = []
for label in label_columns:
    gridCV = GridSearchCV(estimator=linearSVC, param_grid=param_grid, scoring='accuracy',cv=kf)
    gridCV.fit(np_train_data_features, np_train_data_targets[label])
    print("The best parameters for label %s are %s with a score of %0.2f"
      % (label, gridCV.best_params_, gridCV.best_score_))
    best_Cs.append(gridCV.best_params_['C'])

The best parameters are {'C': 100.0} with a score of 0.74
The best parameters are {'C': 0.1} with a score of 0.69
The best parameters are {'C': 1000000.0} with a score of 0.67


In [91]:
'''
1 b (iii)
The best parameters are {'C': 100.0} with a score of 0.74
The best parameters are {'C': 0.1} with a score of 0.69
The best parameters are {'C': 1000000.0} with a score of 0.67
'''
#best_Cs = [100, 0.1, 1000000]
test_pred_labels = []
for i in range(len(label_columns)):
    label = label_columns[i]
    best_C = best_Cs[i]
    linearSvm = LinearSVC(penalty='l1', dual=False, tol=1e-3, max_iter = 100, C=best_C)
    linearSvm.fit(np_train_data_features, np_train_data_targets[label])
    temp_score = linearSvm.score(np_test_data_features, np_test_data_targets[label])
    test_pred_labels.append(linearSvm.predict(np_test_data_features))
    print("Test accuracy for label %s is %0.4f" % (label, temp_score))

test_sample_count = len(test_pred_labels[0])
false_count = 0
for i in range(test_sample_count):
    all_true = True 
    for j in range(len(label_columns)):
        label = label_columns[j]
        if(test_pred_labels[j][i] != np_test_data_targets[label][i]):
            all_true = False
    if not all_true:
        false_count += 1
temp_test_targets = []
for label in label_columns:
    temp_test_targets.append (np_test_data_targets[label])
hamming_loss_rate = np.sum(np.not_equal(np.transpose(temp_test_targets),
                        np.transpose(test_pred_labels)))/float(np.transpose(temp_test_targets).size)
print ("Exact match rate is %0.4f" % ((test_sample_count-false_count)/test_sample_count))
print("hamming loss rate is %0.4f" % (hamming_loss_rate))

Test accuracy for label Family is 0.9390
Test accuracy for label Genus is 0.9276
Test accuracy for label Species is 0.9601
Exact match rate is 0.9019
hamming loss rate is 0.0578


In [125]:
'''
1 b (iv)
'''
from sklearn.utils import resample
species_frequencies = []
species_data = []
for species, data_species in data.groupby('Species'):
    species_data.append (data_species)
    species_frequencies.append(data_species.shape[0])
median_species_frequency = int(np.median(species_frequencies))
mean_species_frequency = int(np.mean(species_frequencies))
balanced_species_data = []
#Class balancing 
#upsampling rare species class by and downsampling frequent frequent species by factor of 2 
for i in range(len(species_data)):
     
    n_samples = int(2*species_data[i].shape[0]) if species_data[i].shape[0] < mean_species_frequency else int(2*species_data[i].shape[0])
    species_data_resampled = resample(species_data[i], replace=True, 
                                      n_samples=n_samples, random_state=1)
    balanced_species_data.append(species_data_resampled)

balanced_species_data = pd.concat(balanced_species_data)


msk = np.random.rand(len(balanced_species_data)) < 0.7
balanced_train_data = balanced_species_data[msk]
# for species, data_species in balanced_train_data.groupby('Genus'):
#     print(data_species.shape[0])
  
np_balanced_train_data_features = balanced_train_data.iloc[:,:22].values
balanced_test_data = balanced_species_data[~msk]
np_balanced_test_data_features = balanced_test_data.iloc[:,:22].values

np_balanced_train_data_targets = {}
np_balanced_test_data_targets = {}
for i in range(3):
    temp_np_train_target = balanced_train_data.iloc[:,22+i:22+i+1].values.ravel()
    temp_np_test_target = balanced_test_data.iloc[:,22+i:22+i+1].values.ravel()
    np_balanced_train_data_targets[label_columns[i]] = temp_np_train_target
    np_balanced_test_data_targets[label_columns[i]] = temp_np_test_target


In [126]:
'''
1 b (iv) conti
'''
PENALTY_PRAMA_POINTS = 10
C_range = np.logspace(-3, 6, PENALTY_PRAMA_POINTS)
linearSVC = LinearSVC(penalty='l1', dual=False, tol=1e-3, max_iter = 100)
kf = KFold(n_splits=10)
param_grid = dict(C=C_range)
best_Cs = []
for label in label_columns:
    gridCV = GridSearchCV(estimator=linearSVC, param_grid=param_grid, scoring='accuracy',cv=kf)
    gridCV.fit(np_balanced_train_data_features, np_balanced_train_data_targets[label])
    print("The best parameters for label %s are %s with a score of %0.2f"
      % (label, gridCV.best_params_, gridCV.best_score_))
    best_Cs.append(gridCV.best_params_["C"])

test_pred_labels = []
for i in range(len(label_columns)):
    label = label_columns[i]
    best_C = best_Cs[i]
    linearSvm = LinearSVC(penalty='l1', dual=False, tol=1e-3, max_iter = 100, C=best_C)
    linearSvm.fit(np_balanced_train_data_features, np_balanced_train_data_targets[label])
    temp_score = linearSvm.score(np_balanced_test_data_features, np_balanced_test_data_targets[label])
    test_pred_labels.append(linearSvm.predict(np_balanced_test_data_features))
    print("Test accuracy for label %s is %0.4f" % (label, temp_score))

test_sample_count = len(test_pred_labels[0])
false_count = 0
for i in range(test_sample_count):
    all_true = True 
    for j in range(len(label_columns)):
        label = label_columns[j]
        if(test_pred_labels[j][i] != np_balanced_test_data_targets[label][i]):
            all_true = False
    if not all_true:
        false_count += 1
temp_test_targets = []
for label in label_columns:
    temp_test_targets.append (np_balanced_test_data_targets[label])
hamming_loss_rate = np.sum(np.not_equal(np.transpose(temp_test_targets),
                        np.transpose(test_pred_labels)))/float(np.transpose(temp_test_targets).size)
print ("Exact match rate is %0.4f" % ((test_sample_count-false_count)/test_sample_count))
print("hamming loss rate is %0.4f" % (hamming_loss_rate))

The best parameters are {'C': 10000.0} with a score of 0.79
The best parameters are {'C': 100.0} with a score of 0.75
The best parameters are {'C': 1000.0} with a score of 0.75
Test accuracy for label Family is 0.9366
Test accuracy for label Genus is 0.9463
Test accuracy for label Species is 0.9593
Exact match rate is 0.9156
hamming loss rate is 0.0526


In [12]:
'''
2 a
'''
import time
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn import metrics


start = time.time()
data_raw = pd.read_csv('Frogs_MFCCs.csv', sep=",", header=0)
data = data_raw.iloc[:,:25]
np_data_features = data.iloc[:,:22].values
np.random.seed(42)
# monte carlo simulation
hamming_distances = []
for x in range(50): 
    #best_CH_index = 0 
    best_silhouette_score = -1 
    #best_k_CH = 0
    best_k_silhouette = 0
    for k in range(2, 50, 1):
        kmeans = KMeans(n_clusters=k).fit(np_data_features)
        labels = kmeans.labels_
    #     chIndex = metrics.calinski_harabaz_score(np_data_features, labels)
    #     if (chIndex > best_CH_index):
    #         best_k = k
    #         best_CH_index = chIndex

        silhouette_score = metrics.silhouette_score(np_data_features, labels)
        if (silhouette_score > best_silhouette_score):
            best_k_silhouette = k
            best_silhouette_score = silhouette_score
    #print ("best CH index is",best_CH_index, "achieved at k =", best_k)   
    #print ("best silhouette_score is",best_silhouette_score, "achieved at k =", best_k_silhouette)  
    
    '''
    2 b, c
    '''
    label_columns = ["Family","Genus","Species"]
    kmeans = KMeans(n_clusters=best_k_silhouette).fit(np_data_features)
    labels = kmeans.labels_


    majority_triplet = {}
    for i in range(len(label_columns)):
        # {cluster0 : {label1: frequency(int), label2: frequency(int)...}}
        label_frequency = {}
        targets = data.iloc[:,22+i:22+i+1].values.tolist()
        for j in range(len(targets)): 
            cluster_number = labels[j]
            target = targets[j][0]
            if cluster_number not in label_frequency:
                label_frequency[cluster_number] = {}
            if target not in label_frequency[cluster_number]: 
                label_frequency[cluster_number][target] = 0 
            label_frequency[cluster_number][target] += 1 
        #print(label_frequency)
        #find the most frequent lable in each cluster and assign that label to the cluster 
        for key in label_frequency: 
            if key not in majority_triplet: 
                majority_triplet[key] = ["", "" ,""]
            majority_name = ""
            majority_vote = 0
            for name in label_frequency[key]: 
                if label_frequency[key][name] > majority_vote:
                    majority_vote = label_frequency[key][name]
                    majority_name = name
            majority_triplet[key][i] = majority_name
    #print (majority_triplet)
    predicted_labels = []
    for i in range(len(targets)):
        cluster_number = labels[i]
        predicted_label = majority_triplet[cluster_number]
        predicted_labels.append(predicted_label)

    np_predicted_labels = np.array(predicted_labels)
    np_target_labels = data[label_columns].values
    hamming_loss_rate = np.sum(np.not_equal(np.transpose(np_target_labels),
                            np.transpose(np_predicted_labels)))/float(np.transpose(np_target_labels).size)
    hamming_score = 1 - hamming_loss_rate
    hamming_distance = hamming_loss_rate * 3
    hamming_distances.append(hamming_distance)
    #print("hamming loss rate is %0.4f\nhamming score is %0.4f\nhamming distance is %0.4f" 
   #       % (hamming_loss_rate, hamming_score, hamming_distance))
    
#end = time.time()
#print("time taken:", end - start)
#print (hamming_distances)

[0.66726893676164, 0.66726893676164, 0.66726893676164, 0.66726893676164, 0.66726893676164, 0.66726893676164, 0.6657400972897846, 0.7000694927032662, 0.6653231410701876, 0.6674079221681724, 0.66726893676164, 0.7021542738012508, 0.7353717859624739, 0.66726893676164, 0.6674079221681724, 0.66726893676164, 0.66726893676164, 0.66726893676164, 0.66726893676164, 0.6674079221681724, 0.66726893676164, 0.66726893676164, 0.66726893676164, 0.66726893676164, 0.6668519805420431, 0.66726893676164, 0.7357887421820708, 0.66726893676164, 0.66726893676164, 0.7000694927032662, 0.66726893676164, 0.66726893676164, 0.66726893676164, 0.66726893676164, 0.66726893676164, 0.66726893676164, 0.66726893676164, 0.66726893676164, 0.66726893676164, 0.66726893676164, 0.66726893676164, 0.66726893676164, 0.66726893676164, 0.7000694927032662, 0.66726893676164, 0.66726893676164, 0.66726893676164, 0.66726893676164, 0.7357887421820708, 0.7353717859624739]


In [15]:
np_hamming_distances = np.array(hamming_distances)
print("average hamming distance is:", np.mean(np_hamming_distances))
print("standard deviation of hamming distance is:", np.std(np_hamming_distances))


average hamming distance is: 0.6753300903405142
standard deviation of hamming distance is: 0.01993378297367031
